# Extractive Summarization with DataLoader

## imports

In [1]:
import json
import numpy as np
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, random_split

from sentence_transformers import models
from sentence_transformers import SentenceTransformer

# from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv

from sklearn.metrics import pairwise_distances

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## dataset

In [53]:
class SummaryDataset(Dataset):
    
    def __init__(self, path):
        
        with open(path, 'r') as f:
            self.data = [json.loads(line) for line in f]
        
    def __len__(self):
        """Returns the number of data."""
        return len(self.data)
    
    def __getitem__(self, idx):
        sentences = self.data[idx]['doc'].split('\n')
        labels = self.data[idx]['labels'].split('\n')
        labels = [int(label) for label in labels]
        print(len(labels))
#         labels = torch.tensor(labels)
        
        return sentences, labels

## dataset split (train, valid, test)

In [54]:
data_path = '../../data/summary/data/train.json'

dataset = SummaryDataset(data_path)

In [55]:
train_size = int(0.6 * len(dataset))
valid_size = int(0.2 * len(dataset))
test_size = len(dataset) - (train_size + valid_size)

train_dataset, valid_dataset, test_dataset \
    = random_split(dataset, [train_size, valid_size, test_size])

In [56]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [57]:
for batch in train_dataloader:
    data, labels = batch
    break

33
36
30
19


In [59]:
len(data)

19

In [20]:
# sample

## Extractive Summarization architecture

### 1) GAT Classifier

In [8]:
class GATClassifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads, num_classes=2):
        super().__init__()
        
        self.out_head = 1
        self.out_dim = out_dim
        
        self.conv1 = GATConv(in_dim, hidden_dim, heads=num_heads, dropout=0.6)
        self.conv2 = GATConv(hidden_dim * num_heads, out_dim, concat=False,
                             heads=self.out_head, dropout=0.6)
        
        self.lstm = nn.LSTM(out_dim, 32, 1, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(32, num_classes)
        
    
    def init_hidden(self, batch_size):
        # (num_layers * num_directions, batch_size, hidden_size)
        hidden = Variable(torch.zeros(1, batch_size, 32))
        cell = Variable(torch.zeros(1, batch_size, 32))
        return hidden, cell
    

    def forward(self, features, edge_index):
        x = F.dropout(features, p=0.6, training=True)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=True)
        x = self.conv2(x, edge_index)
        x = x.view(-1, x.size(0), self.out_dim)
        
        h_0, cell = self.init_hidden(x.size(0))  # initial h_0
        
        output, h_n = self.lstm(x, (h_0, cell))
        
        # many-to-many
        output = self.fc(output)
        
        return output

### 2) Summarizer

In [9]:
class Summarizer(nn.Module):
    
    def __init__(self, 
                 in_dim, 
                 hidden_dim, 
                 out_dim, 
                 num_heads, 
                 num_classes=2):
        super(Summarizer, self).__init__()
        
        self.embedder = SentenceTransformer('bert-base-nli-stsb-mean-tokens')
        self.gat_classifier = GATClassifier(in_dim, hidden_dim, out_dim, num_heads, num_classes)
        
    def build_graph(self, features, threshold=0.2):
        cosine_matrix = 1 - pairwise_distances(features, metric="cosine")
        adj_matrix = (cosine_matrix > threshold) * 1
        
        G = nx.from_numpy_matrix(adj_matrix)
        
        e1_list = [e1 for e1, _ in list(G.edges)]
        e2_list = [e2 for _, e2 in list(G.edges)]
        edge_index = [e1_list, e2_list]
        edge_index = torch.tensor(edge_index)
        
        return edge_index
    
    def forward(self, sents):
        features = self.embedder.encode(sents)
        features = np.array(features)
        
        edge_index = self.build_graph(features)
        features = torch.from_numpy(features)
        
        output = self.gat_classifier(features, edge_index)
        return output

In [12]:
model = Summarizer(in_dim=768,
                   hidden_dim=128,
                   out_dim=64,
                   num_heads=2,
                   num_classes=1)

model = model.to(device)

In [27]:
model.train()
for batch_idx, (data, labels) in enumerate(train_dataloader):
    data, labels = data.to(device), labels.to(device)
    
    optimizer.zero_grad()
    logits = model(data)
    
    labels = labels.float()
    labels = labels.view(-1, logits.size()[1])
    logits = logits.view(-1, logits.size()[1])

    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 19 and 26 in dimension 1 at /pytorch/aten/src/TH/generic/THTensor.cpp:612